In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install numpy pandas matplotlib yfinance statsmodels scikit-learn tensorflow
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
# Step 1: Load Data
def load_data(ticker, start_date, end_date):
    """
    Fetch stock data from Yahoo Finance
    """
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Step 2: Data Preprocessing
def preprocess_data(data):
    """
    Prepare data for time series analysis
    """
    # Use closing prices
    df = data[['Close']]
    
    # Handle missing values
    df = df.fillna(method='ffill')
    
    # Normalization (for LSTM)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)
    
    return df, scaler, scaled_data

In [3]:
# Step 3: Data Visualization
def plot_stock_data(df):
    """
    Plot historical closing prices
    """
    plt.figure(figsize=(12, 6))
    plt.plot(df['Close'], label='Close Price')
    plt.title('Historical Stock Price')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.savefig('stock_prices.png')
    plt.close()

# Step 4: ARIMA Model
def arima_model(df):
    """
    Implement ARIMA model for time series forecasting
    """
    # Split data
    train_size = int(len(df) * 0.8)
    train, test = df[:train_size], df[train_size:]
    
    # Fit ARIMA model
    model = ARIMA(train, order=(5,1,0))  # (p,d,q) parameters
    model_fit = model.fit()
    
    # Forecast
    arima_forecast = model_fit.forecast(steps=len(test))
    
    # Evaluate
    mse = mean_squared_error(test, arima_forecast)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test, arima_forecast)
    
    print(f"ARIMA Metrics:\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}")
    
    return arima_forecast

# Step 5: LSTM Model
def lstm_model(scaled_data, scaler, time_steps=60):
    """
    Implement LSTM model for time series forecasting
    """
    # Create time series dataset
    X, y = [], []
    for i in range(time_steps, len(scaled_data)):
        X.append(scaled_data[i-time_steps:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    
    # Reshape data for LSTM [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Build LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
    
    # Make predictions
    lstm_pred = model.predict(X_test)
    lstm_pred = scaler.inverse_transform(lstm_pred)
    
    # Prepare test data for comparison
    test_data = scaler.inverse_transform(scaled_data[train_size+time_steps:])
    
    # Evaluate
    mse = mean_squared_error(test_data, lstm_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data, lstm_pred)
    
    print(f"LSTM Metrics:\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}")
    
    return lstm_pred

In [4]:
# Step 6: Main Function
def main():
    # Configuration
    ticker = 'AAPL'  # Example: Apple stock
    start_date = '2010-01-01'
    end_date = '2023-01-01'
    
    # Load and preprocess data
    data = load_data(ticker, start_date, end_date)
    df, scaler, scaled_data = preprocess_data(data)
    
    # Plot historical data
    plot_stock_data(df)
    
    # ARIMA implementation
    arima_pred = arima_model(df)
    
    # LSTM implementation
    lstm_pred = lstm_model(scaled_data, scaler)
    
    # Visualization of results
    plt.figure(figsize=(12, 6))
    plt.plot(df.index[-len(arima_pred):], arima_pred, label='ARIMA Predictions')
    plt.plot(df.index[-len(lstm_pred):], lstm_pred, label='LSTM Predictions')
    plt.plot(df['Close'][-len(lstm_pred):], label='Actual Prices')
    plt.title('Stock Price Predictions')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.savefig('predictions_comparison.png')
    plt.close()

if __name__ == '__main__':
    main()

[*********************100%***********************]  1 of 1 completed
C:\Users\gunja\AppData\Local\Temp\ipykernel_15992\4188672797.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
C:\Users\gunja\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gunja\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gunja\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will b

ARIMA Metrics:
MSE: 3994.63
RMSE: 63.20
MAE: 59.21
Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - loss: 0.0037
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 6.9140e-05
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - loss: 8.2235e-05
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 5.3822e-05
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 5.2473e-05
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 6.2787e-05
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 4.9822e-05
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.7217e-05
Epoch 9/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.8466e-05
Epoch 10/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 4.2780e-05
Epoch 11/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 4.2670e-05
Epoch 12/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 4.8556e-05
Epoch 13/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - loss: 4.5842e-05
Epoch 14/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 